In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
#使用chrome的webdriver
browser = webdriver.Chrome()
#開啟頁面
browser.get('https://decipher.sanger.ac.uk/syndrome/16#genotype/cnv/21/patient-overlap/cnvs')
#如果需要執行完自動關閉，就要加上下面這一行
#browser.close()


In [7]:
def get_cnvs_table(browser, clickable_open_page):
    clickable_open_page.click()
    browser.switch_to.window(browser.window_handles[1])
    xp = "//table[@id='patient-cnvs-table']"
    try:
        element = WebDriverWait(browser, 5).until(
            EC.presence_of_element_located(
                (By.XPATH, 
                 xp
                ))
        )
    except TimeoutException:
#         print('Element not found')
        browser.close()
        browser.switch_to.window(parentWindow)
        return
    else:
        cnvs_table = browser.find_element(By.XPATH, xp)
        df = pd.read_html(cnvs_table.get_attribute('outerHTML'))[0]
#         loc_s = browser.find_element(
#                     By.XPATH, 
#                     "//div[@class='position']/span[@class='start']"
#                 ).text
#         loc_e = browser.find_element(
#                     By.XPATH, 
#                     "//div[@class='position']/span[@class='end']"
#                 ).text

        browser.close()
        browser.switch_to.window(parentWindow)
        return df

In [8]:
def get_location(browser, clickable_open_page):
    
    browser.switch_to.window(browser.window_handles[1])
    try:
        element = WebDriverWait(browser, 100).until(
            EC.presence_of_element_located(
                (By.XPATH, 
                 "//div[@class='position']/span[@class='end']"
                ))
        )
    finally:
#         pass
#         browser.implicitly_wait(30)
        loc_s = browser.find_element(
                    By.XPATH, 
                    "//div[@class='position']/span[@class='start']"
                ).text
        loc_e = browser.find_element(
                    By.XPATH, 
                    "//div[@class='position']/span[@class='end']"
                ).text

        browser.close()
        browser.switch_to.window(parentWindow)
        return (int(loc_s), int(loc_e))

In [9]:
# close the prompt frame
try:
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, "//button[@class='close']")
        )
    )
finally:
    a = browser.find_element(By.XPATH, "//button[@class='close']")
    a.click()
    
# change to 100 a page
try:
    element = WebDriverWait(browser, 60).until(
        EC.presence_of_element_located(
            (By.XPATH, 
             "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/div[@class='panel-footer']/div[@class='row']/div[@class='col-xs-2']/div[@class='dataTables_length']/label/select[@class='form-control input-sm']"
            ))
    )
finally:
    sampleNumList = browser.find_element(
        By.XPATH,
        "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/div[@class='panel-footer']/div[@class='row']/div[@class='col-xs-2']/div[@class='dataTables_length']/label/select[@class='form-control input-sm']")
    sampleNumList.send_keys(100)

In [10]:
start_time = time.time()
global parentWindow
parentWindow = browser.window_handles[0]
# turn to the next page
xp = "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/div[@class='panel-footer']/div[@class='row']/div[@class='col-xs-8 text-center']/div[@class='dataTables_paginate paging_simple_numbers']/ul[@class='pagination']/li/a"
try:
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, xp)
        )
    )
finally:    
    pages = browser.find_elements(By.XPATH, xp)[1:-1]

locs, dfs = [], []
not_opened_elements = {}
for idx_p, page in enumerate(pages):
    page.click()
    noes = []
    # get table
    try:
        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//table[@class='table table-striped table-hover data-table vue-table']")
            )
        )
    finally:    
        target_table = browser.find_element(By.XPATH, "//table[@class='table table-striped table-hover data-table vue-table']")
        # get table
        df = pd.read_html(target_table.get_attribute('outerHTML'))[0]
        dfs.append(df)
    clickable_open_pages = browser.find_elements(
        By.XPATH, 
        "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/table[@class='table table-striped table-hover data-table vue-table']/tbody/tr[@class='clickable open-page']/td[1]")

    for idx, clickable_open_page in enumerate(clickable_open_pages):
        loc_df = get_cnvs_table(browser, clickable_open_page)
        if type(loc_df)!=pd.core.frame.DataFrame:
            print(clickable_open_page.text)
            noes.append(clickable_open_page.text)
        else:
            loc_df = loc_df[loc_df.columns[:2]]
            loc_df['Patient'] = int(clickable_open_page.text)
            loc_df.columns = ['Location', 'class/mean_ratio', 'Patient']
            locs.append(loc_df)
#         break
    not_opened_elements[idx_p] = noes
#         break
# print(len(dfs))
consumed_time = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
print('Done within time %s.' % consumed_time)
print(len(noes))

270971
366562
366621
366775
282618
356289
276506
622
986
265746
400975
251228
394949
394976
395364
396458
397154
397192
397243
398084
395346
258298
296465
Done within time 00:25:46.
3


In [11]:
# store for sure
df = pd.concat(dfs, sort=False, ignore_index=True)
df_loc = pd.concat(locs, ignore_index=True, sort=False)
df.to_pickle('df.csv')
df_loc.to_pickle('df_loc.csv')

In [12]:
# check cell
not_opened_elements
a_test = pd.read_pickle('df.csv')
b_test = pd.read_pickle('df_loc.csv')
b_test.shape

(674, 3)

In [13]:
start_time = time.time()
not_opened_elements_2 = {}

# get the dropped elements
for idx_p, page in enumerate(pages):
    if len(not_opened_elements[idx_p])==0:
        continue
    page.click()
    noes = []
    clickable_open_pages = browser.find_elements(
        By.XPATH, 
        "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/table[@class='table table-striped table-hover data-table vue-table']/tbody/tr[@class='clickable open-page']/td[1]")

    for idx, clickable_open_page in enumerate(clickable_open_pages):
        if clickable_open_page.text in set(not_opened_elements[idx_p]):            
            loc_df = get_cnvs_table(browser, clickable_open_page)
            if type(loc_df)!=pd.core.frame.DataFrame:
                print(clickable_open_page.text)
                noes.append(clickable_open_page.text)
            else:
                loc_df = loc_df[loc_df.columns[:2]]
                loc_df['Patient'] = int(clickable_open_page.text)
                loc_df.columns = ['Location', 'class/mean_ratio', 'Patient']
                locs.append(loc_df)
#         break
    not_opened_elements_2[idx_p] = noes
#         break
# print(len(dfs))
consumed_time = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
print('Done within time %s.' % consumed_time)

Done within time 00:01:06.


In [13]:
start_time = time.time()
not_opened_elements = {}

# get the dropped elements
for idx_p, page in enumerate(pages):
    if len(not_opened_elements_2[idx_p])==0:
        continue
    page.click()
    noes = []
    clickable_open_pages = browser.find_elements(
        By.XPATH, 
        "//div[@class='tab-content']/div[@class='tab-pane block active']/div[@class='dataTables_wrapper vue-table-wrapper']/div[@class='panel panel-default']/table[@class='table table-striped table-hover data-table vue-table']/tbody/tr[@class='clickable open-page']/td[1]")

    for idx, clickable_open_page in enumerate(clickable_open_pages):
        if clickable_open_page.text in set(not_opened_elements_2[idx_p]):            
            loc_df = get_cnvs_table(browser, clickable_open_page)
            if type(loc_df)!=pd.core.frame.DataFrame:
                print(clickable_open_page.text)
                noes.append(clickable_open_page.text)
            else:
                loc_df = loc_df[loc_df.columns[:2]]
                loc_df['Patient'] = int(clickable_open_page.text)
                loc_df.columns = ['Location', 'class/mean_ratio', 'Patient']
                locs.append(loc_df)
#         break
    not_opened_elements[idx_p] = noes
#         break
# print(len(dfs))
consumed_time = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
print('Done within time %s.' % consumed_time)

KeyError: 6

In [14]:
from selenium.common.exceptions import StaleElementReferenceException

In [49]:
# merge data
df = pd.concat(dfs, sort=False, ignore_index=True)
df_loc = pd.concat(locs, ignore_index=True, sort=False)
df_loc['chromosome'] = df_loc.Location.apply(lambda x:f'chr{x.split(":")[0]}')
df_loc['location_start'] = df_loc.Location.apply(lambda x:int(x.split(':')[1].split('-')[0]))
df_loc['location_end'] = df_loc.Location.apply(lambda x:int(x.split(':')[1].split('-')[1]))
print(df_loc.shape)
# df_loc_new = pd.concat(new_locs, ignore_index=True, sort=False)
# df_loc_new['chromosome'] = df_loc_new.Location.apply(lambda x:f'chr{x.split(":")[0]}')
# df_loc_new['location_start'] = df_loc_new.Location.apply(lambda x:int(x.split(':')[1].split('-')[0]))
# df_loc_new['location_end'] = df_loc_new.Location.apply(lambda x:int(x.split(':')[1].split('-')[1]))
df_loc_finale = df_loc
# df_loc_finale = pd.concat([df_loc, df_loc_new], ignore_index=True, sort=False)
df_finale = df.merge(df_loc_finale, on='Patient')
df_finale.shape

(700, 6)


(722, 17)

In [258]:
# check cell
df_finale.loc[df_finale.Patient==396032][['Patient', 'Sex', 'Size', 'Overlap','chromosome','location_start','location_end']]
df_finale.groupby('chromosome').agg({'Patient':['nunique', 'count']})
set(df_finale.Patient.unique())-set(df_finale.loc[df_finale.chromosome=='chr22'].Patient.unique())
len(set(df_finale.loc[df_finale.chromosome=='chr22'].Patient.unique()))
# a = df_finale.groupby('Patient').agg({'chromosome':['unique', 'nunique', 'count']})
# a.loc[('chr22' not in a[('chromosome','unique')].values)]
# a[('chromosome','unique')].values

579

# store data

In [43]:
# df_finale.dropna(subset=['Pathogenicity / Contribution'], inplace=True)
# df_finale = df_finale.loc[(df_finale.chromosome=='chr22')&
#               (df_finale['class/mean_ratio'].apply(lambda x:re.search(r'Deletion+', x)!=None))&
#               (df_finale['Pathogenicity / Contribution'].apply(lambda x:re.search(r'Pathogenic+', x)!=None))]

In [21]:
a = df_finale.loc[(df_finale.chromosome=='chr22')&
                  (df_finale['class/mean_ratio'].apply(lambda x:re.search(r'Deletion+', x)!=None))
                 ][['location_start', 'location_end']].describe()
a.location_start = a.location_start.apply(lambda x:str(round(x,2)))
a.location_end = a.location_end.apply(lambda x:str(round(x,2)))
print(a.to_latex())

\begin{tabular}{lll}
\toprule
{} & location\_start & location\_end \\
\midrule
count &          617.0 &        617.0 \\
mean  &    18621292.79 &  22953644.64 \\
std   &     2625773.51 &   3087755.29 \\
min   &            1.0 &   18966490.0 \\
25\%   &     17925446.0 &   21440514.0 \\
50\%   &     18890162.0 &   21505387.0 \\
75\%   &     18919942.0 &   25975446.0 \\
max   &     51168854.0 &   51208042.0 \\
\bottomrule
\end{tabular}



In [50]:
target_chromosome = 'chr22'
df_finale.loc[(df_finale.chromosome==target_chromosome)&
              (df_finale['class/mean_ratio'].apply(lambda x:re.search(r'Deletion+', x)!=None))
#               &(df_finale['Pathogenicity / Contribution'].apply(lambda x:re.search(r'Pathogenic+', x)!=None))
              ][['Patient', 'chromosome', 'location_start', 'location_end', 
                 'Pathogenicity / Contribution', 'Sex', 'Size', 'Overlap', 
                 'Genes in common', 'Functional Similarity', 'Genotype / Class', 
                 'Inheritance', 'Phenotypes', 'class/mean_ratio']].to_csv(f'./data/spider_res-{target_chromosome}-deletion.csv', sep=',', index=False)

In [310]:
df_finale.loc[(df_finale.chromosome=='chr22')&
              (df_finale['class/mean_ratio'].apply(lambda x:re.search(r'Deletion+', x)!=None))&
              (df_finale['Pathogenicity / Contribution'].apply(lambda x:re.search(r'Pathogenic+', x)!=None))
             ][['Patient', 'chromosome', 'location_start', 'location_end', 
                'Pathogenicity / Contribution', 'Sex', 'Size', 'Overlap', 
                'Genes in common', 'Functional Similarity', 'Genotype / Class', 
                'Inheritance', 'Phenotypes', 'class/mean_ratio']].describe().to_csv('./data/description-v1.csv', sep=',')

In [296]:
df_finale.to_csv('./data/spider_res-chr15.csv', sep=',', index=False)

In [224]:
df_finale.to_pickle('./res_677.pkl')

In [129]:
df_loc = pd.concat(locs, ignore_index=True, sort=False)
df_loc['chromosome'] = df_loc.Location.apply(lambda x:f'chr{x.split(":")[0]}')
df_loc['location_start'] = df_loc.Location.apply(lambda x:int(x.split(':')[1].split('-')[0]))
df_loc['location_end'] = df_loc.Location.apply(lambda x:int(x.split(':')[1].split('-')[1]))
# df_loc.dtypes
dfs = []
for page in pages:
    # get table
    try:
        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//table[@class='table table-striped table-hover data-table vue-table']")
            )
        )
    except TimeoutException:
        print('Element not found')
    else:    
        target_table = browser.find_element(By.XPATH, "//table[@class='table table-striped table-hover data-table vue-table']")
        # get table
        df = pd.read_html(target_table.get_attribute('outerHTML'))[0]
    break
df.dtypes
    

Patient                           int64
Unnamed: 1                      float64
Sex                              object
Size                             object
Overlap                          object
Genes in common                   int64
Functional Similarity           float64
Pathogenicity / Contribution     object
Genotype / Class                 object
Inheritance                      object
Phenotypes                       object
Unnamed: 11                     float64
dtype: object

In [119]:
# test 
a = pd.concat(locs)
a.columns = ['Location', 'class/mean_ratio', 'Patient']
a['chromosome'] = a.Location.apply(lambda x:f'chr{x.split(":")[0]}')
a['location_start'] = a.Location.apply(lambda x:int(x.split(':')[1].split('-')[0]))
a['location_end'] = a.Location.apply(lambda x:int(x.split(':')[1].split('-')[1]))
# a[['Location', 'class/mean_ratio']]
# a.columns
a.head()

,Location,class/mean_ratio,Patient,chromosome,location_start,location_end
0,22:18919742-21440655,Deletion-0.89453,250577,chr22,18919742,21440655
0,22:18919942-21440514,Deletion-1,262079,chr22,18919942,21440514
0,22:18919742-21440655,Deletion-1,262934,chr22,18919742,21440655
0,22:18919942-21440514,Deletion-1,263156,chr22,18919942,21440514
0,16:6704616-6785666,Deletion-1,263159,chr16,6704616,6785666


# test cells

In [77]:
# # test get_cnvs_table
# clickable_open_page.click()
# browser.switch_to.window(browser.window_handles[1])
# xp = "//table[@id='patient-cnvs-table']"
# try:
#     element = WebDriverWait(browser, 100).until(
#         EC.presence_of_element_located(
#             (By.XPATH, 
#              xp
#             ))
#     )
# finally:
#     cnvs_table = browser.find_element(By.XPATH, xp)
#     df_loc = pd.read_html(cnvs_table.get_attribute('outerHTML'))[0]
# #         loc_s = browser.find_element(
# #                     By.XPATH, 
# #                     "//div[@class='position']/span[@class='start']"
# #                 ).text
# #         loc_e = browser.find_element(
# #                     By.XPATH, 
# #                     "//div[@class='position']/span[@class='end']"
# #                 ).text

# browser.close()
browser.switch_to.window(parentWindow)